In [165]:
# module insatall
import torch
import torch.nn as nn
import torch.optim as opt
import torch.optim.lr_scheduler as lrs
import torch.utils.data as dat
import torch.nn.utils as utils
from torch.nn.utils.rnn import pad_sequence
# distributed learning
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
# a pytorch extension fp-16
from apex.parallel import DistributedDataParallel as DDP_APEX
from apex import amp
# gene batch loader
from file import GeneBatchLoader, sanitize, vocab, write_fna_faa
# fetch all
from database import fetchall

# train test split
from sklearn.model_selection import KFold, StratifiedKFold

# base package
import re
import glob
import os
import time
import numpy as np
from tqdm import tqdm
from Bio import SeqIO

In [137]:
#file = open('./Convertdata/Bacillus_subtilis_168_translated_cds.faa-GCF_002119445.1_ASM211944v1_translated_cds.faa')

In [155]:
file = open('./Convertdata/sample.faa')

In [156]:
thudata = []
subdata = []
for line in file:
    thu = ''
    sub = ''
    tabct = 0
    cp = 0
    for i in range(0, len(line)):
        if(line[i]=='\t'):
            tabct+=1
            if(tabct==2):
                cp=1
            if(tabct==3):
                cp=1
        if(tabct==2):
            if(line[i]==' '):
                cp=0
            if(cp==1):
                sub += line[i]
        if(tabct==3):
            if(line[i]==' '):
                cp=0
            if(cp==1):
                thu += line[i]
    thudata.append(thu)
    subdata.append(sub)
print(thudata)
print(subdata)

['\tlcl|NZ_CP021061.1_prot_WP_001133932.1_2128', '\tlcl|NZ_CP021061.1_prot_WP_000060002.1_3663', '\tlcl|NZ_CP021061.1_prot_WP_000730788.1_1924', '\tlcl|NZ_CP021061.1_prot_WP_000503031.1_2277', '\tlcl|NZ_CP021061.1_prot_WP_000164055.1_3798', '\tlcl|NZ_CP021061.1_prot_WP_000999407.1_3374', '\tlcl|NZ_CP021061.1_prot_WP_001126128.1_3734', '\tlcl|NZ_CP021061.1_prot_WP_000412775.1_3025', '\tlcl|NZ_CP021061.1_prot_WP_000579691.1_5415', '\tlcl|NZ_CP021061.1_prot_WP_000045587.1_5042', '\tlcl|NZ_CP021061.1_prot_WP_000072222.1_4315', '\tlcl|NZ_CP021061.1_prot_WP_000455951.1_3743', '\tlcl|NZ_CP021061.1_prot_WP_000238555.1_3417', '\tlcl|NZ_CP021061.1_prot_WP_000009460.1_4606', '\tlcl|NZ_CP021061.1_prot_WP_000811849.1_4248', '\tlcl|NZ_CP021061.1_prot_WP_001282866.1_5373', '\tlcl|NZ_CP021061.1_prot_WP_000572301.1_938', '\tlcl|NZ_CP021061.1_prot_WP_000026972.1_2968', '\tlcl|NZ_CP021061.1_prot_WP_000478997.1_3693', '\tlcl|NZ_CP021061.1_prot_WP_001033497.1_5317', '\tlcl|NZ_CP021061.1_prot_WP_000579372.1

In [157]:
for i in range(len(thudata)):
    thudata[i]= thudata[i].replace('\t', '')
    thudata[i]= thudata[i].replace('prot', 'cds')
print(thudata)

['lcl|NZ_CP021061.1_cds_WP_001133932.1_2128', 'lcl|NZ_CP021061.1_cds_WP_000060002.1_3663', 'lcl|NZ_CP021061.1_cds_WP_000730788.1_1924', 'lcl|NZ_CP021061.1_cds_WP_000503031.1_2277', 'lcl|NZ_CP021061.1_cds_WP_000164055.1_3798', 'lcl|NZ_CP021061.1_cds_WP_000999407.1_3374', 'lcl|NZ_CP021061.1_cds_WP_001126128.1_3734', 'lcl|NZ_CP021061.1_cds_WP_000412775.1_3025', 'lcl|NZ_CP021061.1_cds_WP_000579691.1_5415', 'lcl|NZ_CP021061.1_cds_WP_000045587.1_5042', 'lcl|NZ_CP021061.1_cds_WP_000072222.1_4315', 'lcl|NZ_CP021061.1_cds_WP_000455951.1_3743', 'lcl|NZ_CP021061.1_cds_WP_000238555.1_3417', 'lcl|NZ_CP021061.1_cds_WP_000009460.1_4606', 'lcl|NZ_CP021061.1_cds_WP_000811849.1_4248', 'lcl|NZ_CP021061.1_cds_WP_001282866.1_5373', 'lcl|NZ_CP021061.1_cds_WP_000572301.1_938', 'lcl|NZ_CP021061.1_cds_WP_000026972.1_2968', 'lcl|NZ_CP021061.1_cds_WP_000478997.1_3693', 'lcl|NZ_CP021061.1_cds_WP_001033497.1_5317', 'lcl|NZ_CP021061.1_cds_WP_000579372.1_5062', 'lcl|NZ_CP021061.1_cds_WP_001104773.1_3720', 'lcl|NZ_CP

In [158]:
for i in range(len(subdata)):
    subdata[i]= subdata[i].replace('\t', '')
    subdata[i]= subdata[i].replace('prot', 'cds')
print(subdata)

['lcl|NC_000964.3_cds_NP_391076.3_3368', 'lcl|NC_000964.3_cds_NP_389540.1_1734', 'lcl|NC_000964.3_cds_NP_391609.2_3927', 'lcl|NC_000964.3_cds_NP_389714.1_1921', 'lcl|NC_000964.3_cds_NP_389369.1_1560', 'lcl|NC_000964.3_cds_NP_390600.1_2880', 'lcl|NC_000964.3_cds_NP_389435.1_1627', 'lcl|NC_000964.3_cds_NP_388606.1_770', 'lcl|NC_000964.3_cds_NP_387936.1_55', 'lcl|NC_000964.3_cds_NP_391396.1_3699', 'lcl|NC_000964.3_cds_NP_390687.2_2972', 'lcl|NC_000964.3_cds_NP_389426.2_1618', 'lcl|NC_000964.3_cds_NP_389683.1_1885', 'lcl|NC_000964.3_cds_NP_390910.1_3199', 'lcl|NC_000964.3_cds_NP_390618.1_2899', 'lcl|NC_000964.3_cds_NP_387888.1_7', 'lcl|NC_000964.3_cds_NP_388944.2_1106', 'lcl|NC_000964.3_cds_NP_388065.2_185', 'lcl|NC_000964.3_cds_NP_389476.2_1669', 'lcl|NC_000964.3_cds_NP_388553.2_714', 'lcl|NC_000964.3_cds_NP_391410.1_3714', 'lcl|NC_000964.3_cds_NP_389448.1_1640', 'lcl|NC_000964.3_cds_NP_390787.1_3075', 'lcl|NC_000964.3_cds_NP_388530.2_690', 'lcl|NC_000964.3_cds_NP_389551.1_1745', 'lcl|NC_

In [164]:
for i in subdata:
    print(i)

lcl|NC_000964.3_cds_NP_391076.3_3368
lcl|NC_000964.3_cds_NP_389540.1_1734
lcl|NC_000964.3_cds_NP_391609.2_3927
lcl|NC_000964.3_cds_NP_389714.1_1921
lcl|NC_000964.3_cds_NP_389369.1_1560
lcl|NC_000964.3_cds_NP_390600.1_2880
lcl|NC_000964.3_cds_NP_389435.1_1627
lcl|NC_000964.3_cds_NP_388606.1_770
lcl|NC_000964.3_cds_NP_387936.1_55
lcl|NC_000964.3_cds_NP_391396.1_3699
lcl|NC_000964.3_cds_NP_390687.2_2972
lcl|NC_000964.3_cds_NP_389426.2_1618
lcl|NC_000964.3_cds_NP_389683.1_1885
lcl|NC_000964.3_cds_NP_390910.1_3199
lcl|NC_000964.3_cds_NP_390618.1_2899
lcl|NC_000964.3_cds_NP_387888.1_7
lcl|NC_000964.3_cds_NP_388944.2_1106
lcl|NC_000964.3_cds_NP_388065.2_185
lcl|NC_000964.3_cds_NP_389476.2_1669
lcl|NC_000964.3_cds_NP_388553.2_714
lcl|NC_000964.3_cds_NP_391410.1_3714
lcl|NC_000964.3_cds_NP_389448.1_1640
lcl|NC_000964.3_cds_NP_390787.1_3075
lcl|NC_000964.3_cds_NP_388530.2_690
lcl|NC_000964.3_cds_NP_389551.1_1745
lcl|NC_000964.3_cds_NP_390698.1_2984
lcl|NC_000964.3_cds_NP_390638.2_2920
lcl|NC_000

In [181]:
f = open( './thudata/thuconv.fna')
for i in f:
    print(i)

1

ATGCCTAACAGTCAAAAGATTCGTCATTCGTTATCATCTGCGCAGTCTGGTATGTGGTTTGCGCAACAATTAGATCCGCT

TAATCCGATATATAATACTGGGGAATATGTAGAAATAAACGGAAATATTTATCAAGACATTTTTGAATTAGCTGTACGTA

AAGTTGTAAGGGAAGCAGAGGCACTTCACGTCCGCTTCGAAGAGGATGAAATTGGCCCATGGCAAGTTGTTGAAGAATCA

CAGTTTCATATGCATTTTATTGACGTTCGTAAAGAAGAAAACCCTGAAGAAGCTGCTAAAGTATGGATGAAAAATGATTT

ATCTATACCAGTCAATTTGAAAAAAGATATATTATTTACTGAAGCACTTATTCAAGTTGAAAATAATCGTTTCTTTTGGT

ATCAGCGCATCCATCACATTGTGATGGATGGCTATGGATTTTCACTTCTTAGCCAAAAGGTAGCGAATGAGTATACATCA

CTTATAGAAGAAACAAACAAGAGTGAAAAGCCATTTGGTTCTCTTGCAAAAGTCGTGCAAGAAGATATCGAGTATCGTGA

GTCTAAACAATTTCAGGAAAATCGTACTTTTTGGTTAGAAAAGTTTGCAGATGCCCCAGAAGTTGTAAGTTTGGCAGAAC

GAGCACCGAGAACATCAAATGGATTTTCACGAGAAACAGCTTATTTATCTAGTTCTAGCACGAAAACTTTACTAGAAGAT

ATCAATATTTCGTTAACAAGTTGGCCGGAATTCATTGTGGCTGTAACAAGTATTTATATGCATAAATTAACAGGTGAAAA

TGATATTGTTTTAGGTTTACCGATGATGGGACGTCTAGGTTCTGTATCTATTCATACACCAAGTATGGTAATGAATTTAG

TTCCACTTCGTATTACAGTAACTCCAAATATAACTCTTGCGGAATTATTGCAGCAAGTTTCTAAAGAAATCCGAGATGTA

CGTCGCCATTATA

In [182]:
for line in f:
    for i in range(0, len(line)):
        if(line[0] == 'A'):
            continue
        if(line[0] == 'T'):
            continue
        if(line[0] == 'G'):
            continue
        if(line[0] == 'C'):
            continue
        else:
            line = '>'  + line[0]

In [180]:
for i in f:
    print(i)